In [1]:
#!/usr/local/bin/python3

import os
import sys
from time import time
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import roc_curve, auc, log_loss
from scipy.stats import randint as sp_randint
from scipy import interp
from drivendata_validator import DrivenDataValidator
import itertools
from tpot import TPOTRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# data directory
DATA_DIR = os.path.join('..', 'data')

In [2]:
## Make Submission DataFrame
def make_country_sub(preds, test_feat, country):
    # make sure we code the country correctly
    country_codes = ['A', 'B', 'C']
    
    # get just the poor probabilities
    country_sub = pd.DataFrame(data=preds[:, 1],  # proba p=1
                               columns=['poor'], 
                               index=test_feat.index)

    
    # add the country code for joining later
    country_sub["country"] = country
    return country_sub[["country", "poor"]]

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

##### Household-level survey data: 
This is obfuscated data from surveys conducted by The World Bank, focusing on household-level statistics. The data come from three different countries, and are separated into different files for convenience.

##### Individual-level survey data: 
This is obfuscated data from related surveys conducted by The World Bank, only these focus on individual-level statistics. The set of interviewees and countries involved are the same as the household data, as indicated by shared id indices, but this data includes detailed (obfuscated) information about household members.

##### Submission format:
This gives us the filenames and columns of our submission prediction, filled with all 0.5 as a baseline.

In [3]:
## define data paths
data_paths = {'train_x': os.path.join(DATA_DIR, 'dengue_features_train.csv'),
              'train_y': os.path.join(DATA_DIR, 'dengue_labels_train.csv'),
               'test_x':  os.path.join(DATA_DIR, 'dengue_features_test.csv')}

In [4]:
# load training data
X_train = pd.read_csv(data_paths['train_x'])
y_train = pd.read_csv(data_paths['train_y'])
X_train.drop(columns='week_start_date', inplace=True)

# load test data
X_test = pd.read_csv(data_paths['test_x'])
X_test.drop(columns='week_start_date', inplace=True)

#### The first thing to notice is that each country's surveys have wildly different numbers of columns, so we'll plan on training separate models for each country and combining our predictions for submission at the end.

### Pre-process Data

In [5]:
print("Shapes before transformation")
print("Train : ", X_train.shape)
print("Train Labels : ", y_train.shape)
print("Test : ", X_test.shape)
print("Columns : ", X_train.columns)
train_data = pd.merge(X_train, y_train, on=['city', 'year', 'weekofyear'])
train_data.index = np.arange(0, train_data.shape[0])
X_test.index = np.arange(train_data.shape[0]+1, train_data.shape[0]+X_test.shape[0]+1)

Shapes before transformation
('Train : ', (1456, 23))
('Train Labels : ', (1456, 4))
('Test : ', (416, 23))
('Columns : ', Index([u'city', u'year', u'weekofyear', u'ndvi_ne', u'ndvi_nw', u'ndvi_se',
       u'ndvi_sw', u'precipitation_amt_mm', u'reanalysis_air_temp_k',
       u'reanalysis_avg_temp_k', u'reanalysis_dew_point_temp_k',
       u'reanalysis_max_air_temp_k', u'reanalysis_min_air_temp_k',
       u'reanalysis_precip_amt_kg_per_m2',
       u'reanalysis_relative_humidity_percent',
       u'reanalysis_sat_precip_amt_mm',
       u'reanalysis_specific_humidity_g_per_kg', u'reanalysis_tdtr_k',
       u'station_avg_temp_c', u'station_diur_temp_rng_c',
       u'station_max_temp_c', u'station_min_temp_c', u'station_precip_mm'],
      dtype='object'))


In [6]:
def pre_process_train_test_data(train, test, label_var, exclude_scaling):
    labels = np.ravel(train[label_var])
    train = pd.get_dummies(train.drop(label_var, axis=1))
    test = pd.get_dummies(test)

    # match test set and training set columns
    to_drop = np.setdiff1d(test.columns, train.columns)
    to_add = np.setdiff1d(train.columns, test.columns)

    test.drop(to_drop, axis=1, inplace=True)
    test = test.assign(**{c: 0 for c in to_add})

    train.fillna(0, inplace=True)
    test.fillna(0, inplace=True)

    test_indices = test.index
    train_indices = train.index
    train_test = pd.concat([train, test])
    train_test.sort_values(['year', 'weekofyear'], inplace=True)
    train_test.interpolate(method='linear', inplace=True)

    print("Shapes before transformation")
    print("Train : ", train.shape)
    print("Test : ", test.shape)
    print("Train + Test : ", train_test.shape)

    numeric_vals = train_test.select_dtypes(include=['int64', 'float64'])
    numeric_vals = numeric_vals.loc[:, [x for x in list(numeric_vals.columns.values) if x not in exclude_scaling]]
    scaler = StandardScaler()
    train_test[numeric_vals.columns] = scaler.fit_transform(numeric_vals)

    train = train_test.loc[train_indices, :]
    test = train_test.loc[test_indices, :]

    train[label_var] = labels

    print("Shapes after transformation")
    print("Train : ", train.shape)
    print("Test : ",  test.shape)

    return train, test


print("Preprocessing Training")
label_var = 'total_cases'
exclude_scaling = ['year', 'weekofyear']
a_train, a_test = pre_process_train_test_data(train_data, X_test, label_var, exclude_scaling)
X_train = a_train.drop(label_var, axis=1)
y_train = np.ravel(a_train[label_var])

Preprocessing Training
Shapes before transformation
('Train : ', (1456, 24))
('Test : ', (416, 24))
('Train + Test : ', (1872, 24))
Shapes after transformation
('Train : ', (1456, 25))
('Test : ', (416, 24))


In [7]:
np.setdiff1d(a_test.columns.values, X_train.columns.values)

array([], dtype=object)

In [8]:
## restructure train data
all_train_data = {'features': X_train,
                  'labels': y_train}

## restructure test data
all_test_data = {'features': a_test}

### Cross-validation -- Tune Parameters

In [9]:
X = all_train_data['features'].values.astype(np.float32)
y = all_train_data['labels'].astype(np.int16)
X_test = all_test_data['features'].values.astype(np.float32)

In [15]:
tune_params = 1
if tune_params > 0:
    bestParams = []
    cv = StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=42)
    X = all_train_data['features'].values.astype(np.float32)
    y = all_train_data['labels'].astype(np.int16)
    pipeline_optimizer = TPOTRegressor(scoring='neg_mean_absolute_error', cv=cv, 
                                        periodic_checkpoint_folder='../data/tpot_best_models/',
                                        n_jobs=1, random_state=42, verbosity=3, memory='auto',
                                        generations=1000, max_eval_time_mins=10)
    pipeline_optimizer.fit(X, y)
    pipeline_optimizer.export('tpot_best_model_pipeline.py')

28 operators have been imported by TPOT.


RuntimeError: A pipeline has not yet been optimized. Please call fit() first.

In [16]:
X.shape

(1456, 24)

In [17]:
y.shape

(1456,)

### Training Phase

In [ ]:
# trained_models = {}

# ## Train Model A
# from sklearn.ensemble import GradientBoostingClassifier
# trained_models['A'] = GradientBoostingClassifier(learning_rate=0.1, max_depth=7, 
#                                                  max_features=0.5, min_samples_leaf=14, 
#                                                  min_samples_split=20, n_estimators=100, subsample=0.6)
# trained_models['A'].fit(all_train_data['A']['features'], all_train_data['A']['labels'])

# ## Train Model B
# from sklearn.preprocessing import FunctionTransformer
# from sklearn.pipeline import make_pipeline, make_union
# from sklearn.linear_model import LogisticRegression
# from copy import copy

# trained_models['B'] = make_pipeline(make_union(FunctionTransformer(copy), 
#                                                FunctionTransformer(copy)),
#                                     LogisticRegression(C=0.01, dual=True, penalty="l2"))

# trained_models['B'].fit(all_train_data['B']['features'], all_train_data['B']['labels'])

# ## Train Model C
# from sklearn.naive_bayes import BernoulliNB
# from sklearn.ensemble import RandomForestClassifier
# from tpot.builtins import StackingEstimator

# trained_models['C'] = make_pipeline(
#     make_union(FunctionTransformer(copy), 
#                StackingEstimator(estimator=BernoulliNB(alpha=1.0, fit_prior=False))),
#     RandomForestClassifier(bootstrap=True, criterion="entropy", 
#                            max_features=0.8, min_samples_leaf=6, 
#                            min_samples_split=19, n_estimators=100))

# trained_models['C'].fit(all_train_data['C']['features'], all_train_data['C']['labels'])

### Testing Phase

In [ ]:
# # ## Predict
# predictions = {}
# for grp in all_train_data:
#     predictions[grp] = trained_models[grp].predict_proba(all_test_data[grp]['features'])

### Validate and Submit

In [ ]:
# # convert preds to data frames
# predictionsDF = {}
# for grp in all_train_data:
#     predictionsDF[grp] = make_country_sub(predictions[grp], all_test_data[grp]['features'], grp)

# submission = []
# submission = pd.concat([predictionsDF['A'], predictionsDF['B'], predictionsDF['C']])

# ## Submission Format
# submission.to_csv('../data/my_submission.csv')

# # no parameters unless we have a read_csv kwargs file
# v = DrivenDataValidator()

# if v.is_valid('../data/submission_format.csv', '../data/my_submission.csv'):
#     print "I am awesome."
# else:
#     print "I am not so cool."


In [ ]:
from sklearn.datasets import load_digits

digits = load_digits()